In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
from numpy import save

In [2]:
# Load the data and define the (directed) edges list
G = nx.read_edgelist("BA-100-3-1.txt")
Edges = list(G.edges())
Directed_edges = Edges
for i in range(0, len(Edges)):
    Directed_edges = Directed_edges + [(Edges[i][1], Edges[i][0])]
# print(Directed_edges)
# print(len(Directed_edges))
# print(list(G.edges()))

In [3]:
# N is the number of nodes
# E is the number of edges
N = G.number_of_nodes()
E = len(Edges)

In [4]:
# Define the transition probability matrix

def T(a, b):
    A = np.zeros((2*E, 2*E))
    for i in range(0, 2*E):
        for j in range(0, 2*E):
            if Directed_edges[i][1] == Directed_edges[j][0]:
                if Directed_edges[i][0] == Directed_edges[j][1]:
                    A[i, j] = a
                elif (Directed_edges[i][0], Directed_edges[j][1]) in Directed_edges:
                    A[i, j] = b
                else:
                    A[i, j] = 1
    D = np.sum(A, axis=1)
    D = np.diagflat(D)
    return np.dot(np.linalg.inv(D), A)

In [5]:
M = np.zeros((2*E, 2*E))
for i in range(0, 2*E):
    for j in range(0, 2*E):
        if Directed_edges[j][1] == Directed_edges[i][1]:
            M[i,j] = 1

In [6]:
# Initial condition for Euler method

S0 = [0 for i in range(0, 2*E)]
I0 = [1*N/(2*E) for i in range(0, 2*E)]

In [7]:
# Define Euler method
# h is the stepsize
def euler(h = 0.1, iteration = 1000, beta = 1, a = 1, b = 1, mu = 1, D_S = 1, D_I = 1):
    Y = []
    Z = []
    y = S0
    z = I0
    for n in range(0, iteration):
        tempy = np.array(y)
        tempz = np.array(z)
        y = tempy + (-beta*np.multiply(tempy, np.dot(M, tempz)) + mu*tempz - D_S*tempy + D_S*np.dot(np.transpose(T(a,b)), tempy))*h
        z = tempz + ( beta*np.multiply(tempy, np.dot(M, tempz)) - mu*tempz - D_I*tempz + D_I*np.dot(np.transpose(T(a,b)), tempz))*h
        y = y.tolist()
        z = z.tolist()
        Y.append(y)
        Z.append(z)
        if n>=10 and abs(sum(Z[-1])-sum(Z[-2]))/G.number_of_nodes()<10**(-5):
            break
    return Z

In [8]:
# xx=np.linspace(0, 1.5, 31)
# yy1=[]
# yy2=[]
# yy3=[]
# yy4=[]
# for i in range(0,len(xx)):
#     Result=euler(h = 0.1, iteration = 3000, beta = xx[i], a = 0.1, b = 0.1, mu = 1, D_S = 1, D_I = 1)
#     yy1.append(sum(Result[-1])/G.number_of_nodes())
# for i in range(0,len(xx)):
#     Result=euler(h = 0.1, iteration = 1000, beta = xx[i], a = 5, b = 0.1, mu = 1, D_S = 1, D_I = 1)
#     yy2.append(sum(Result[-1])/G.number_of_nodes())
# for i in range(0,len(xx)):
#     Result=euler(h = 0.1, iteration = 1000, beta = xx[i], a = 0.1, b = 5, mu = 1, D_S = 1, D_I = 1)
#     yy3.append(sum(Result[-1])/G.number_of_nodes())
# for i in range(0,len(xx)):
#     Result=euler(h = 0.1, iteration = 1000, beta = xx[i], a = 5, b = 5, mu = 1, D_S = 1, D_I = 1)
#     yy4.append(sum(Result[-1])/G.number_of_nodes())

# save('yy1-new', np.array(yy1))
# save('yy2-new', np.array(yy2))
# save('yy3-new', np.array(yy3))
# save('yy4-new', np.array(yy4))

# plt.plot(xx, yy1, label ='D=1')
# plt.plot(xx, yy2, label ='D=20')
# plt.plot(xx, yy3, label ='D=0')
# plt.plot(xx, yy4, label ='D=0.1')
# plt.xlabel('Infected rate')
# plt.ylabel('Fraction infected')
# plt.legend()
# plt.show()

In [9]:
# In the following Monte Carlo simulation, to avoid reading function T(a, b) to many times, we restore the transition matrix as T1, T2, T3, T4
T1 = T(0.1, 0.1)
T2 = T(5, 0.1)
T3 = T(0.1, 5)
T4 = T(5, 5)

In [10]:
# # Use the Monte Carlo simulation mentioned in [Colizza 2007 Nature Physics]

# # reaction process
# # dt represents a small time window
# # We assume the recover rate equal to 1, thus mu*dt = dt

# dt = 0.01

# # evaluate the number of infected individuals in each node
# # list "count" represents the number of infected individuals in each node
# count = [0]*100
# for item in arr:
#     count[int(item[0][1])] = count[int(item[0][1])] + 1
    
# for i in range(0, len(arr)):
#     # recover process
#     if arr[i][1] == 'I' and np.random.uniform(0, 1, 1)[0] < dt:
#         arr[i][1] = 'S'
#         count[int(arr[i][0][1])] = count[int(arr[i][0][1])] - 1
#     # infection process
#     # instead of using np.random.uniform(0, 1, 1)[0] < 1 - (1 - beta*dt)**count[int(arr[i][0][1])]
#     # we use np.random.uniform(0, 1, 1)[0] < count[int(arr[i][0][1])]*beta*dt
#     # The correctness is guaranteed by Bernoulli's approximation, i.e., (1+x)^n \approx 1+nx
#     # and we know that dt is small and beta*dt is also small
#     if arr[i][1] == 'S' and np.random.uniform(0, 1, 1)[0] < count[int(arr[i][0][1])]*beta*dt:
#         arr[i][1] = 'I'
#         count[int(arr[i][0][1])] = count[int(arr[i][0][1])] + 1

# # diffusion process
# # We assume the diffusion rate D_I = D_S = 1, thus we do not have to distinguish whether an individual is infected or not.
# # The diffusion rate D_I*dt = D_S*dt = dt
# for i in range(0, len(arr)):
#     if np.random.uniform(0, 1, 1)[0] < dt:
#         # random.choices(Directed_edges, weights = T1[Directed_edges.index(arr[i][0])])[0] represents the next directed edges an individual will visit
#         next_edge = random.choices(Directed_edges, weights = T1[Directed_edges.index(arr[i][0])])[0]
#         arr[i][0] = next_edge
#         count[int(next_edge[0])] = count[int(next_edge[0])] - 1
#         count[int(next_edge[1])] = count[int(next_edge[1])] + 1

In [ ]:
# The cell above is the 'kernel' of this cell, that is, to run the simulation only once
# to run the simulation for 300 seconds, we should add another outer loops
# we let beta from 0.1 to 1.5
# for each beta, run simulation for 100 times
# for each beta and simulation, the reaction-diffusion process exacuted for 30000*dt = 300 seconds

# Use the Monte Carlo simulation mentioned in [Colizza 2007 Nature Physics]

# reaction process
# dt represents a small time window
# We assume the recover rate equal to 1, thus mu*dt = dt

dt = 0.01

Final = []
for beta in np.linspace(0.002, 0.03, 15):
    # Monte Carlo Simulation
    # arr = [[('67', '40'), 'I'], [('15', '71'), 'I'], [('15', '3'), 'I'],...,[('5', '6'), 'I'], [('32', '97'), 'I']]
    # [('67', '40'), 'I'] represents from node '67' to node '40', infected individual
    rho = 50
    arr = [[random.choice(Directed_edges), 'S'] for i in range(0, rho*(N - 1))]
    arr = arr + [[random.choice(Directed_edges), 'I'] for i in range(0, rho)]
#     arr = [[random.choice(Directed_edges), 'I'] for i in range(0, rho*N)]
#     print(arr)
    # print(len(arr))
    
    # evaluate the number of infected individuals in each node
    # list "count" represents the number of infected individuals in each node
    count = [0]*100
    for item in arr:
        if item[1] == 'I':
            count[int(item[0][1])] = count[int(item[0][1])] + 1
            
    for ITERATION in range(0, 100):
        for iteration in range(0, 30000):
            for i in range(0, len(arr)):
                # recover process
                if arr[i][1] == 'I' and np.random.uniform(0, 1, 1)[0] < dt:
                    arr[i][1] = 'S'
                    count[int(arr[i][0][1])] = count[int(arr[i][0][1])] - 1
                # infection process
                # instead of using np.random.uniform(0, 1, 1)[0] < 1 - (1 - beta*dt)**count[int(arr[i][0][1])]
                # we use np.random.uniform(0, 1, 1)[0] < count[int(arr[i][0][1])]*beta*dt
                # The correctness is guaranteed by Bernoulli's approximation, i.e., (1+x)^n \approx 1+nx
                # and we know that dt is small and beta*dt is also small
                if arr[i][1] == 'S' and np.random.uniform(0, 1, 1)[0] < count[int(arr[i][0][1])]*beta*dt:
                    arr[i][1] = 'I'
                    count[int(arr[i][0][1])] = count[int(arr[i][0][1])] + 1

            # diffusion process
            # We assume the diffusion rate D_I = D_S = 1, thus we do not have to distinguish whether an individual is infected or not.
            # The diffusion rate D_I*dt = D_S*dt = dt
            for i in range(0, len(arr)):
                if np.random.uniform(0, 1, 1)[0] < dt:
                    # random.choices(Directed_edges, weights = T1[Directed_edges.index(arr[i][0])])[0] represents the next directed edges an individual will visit
                    next_edge = random.choices(Directed_edges, weights = T1[Directed_edges.index(arr[i][0])])[0]
                    arr[i][0] = next_edge
                    if arr[i][1] == 'I':
                        count[int(next_edge[0])] = count[int(next_edge[0])] - 1
                        count[int(next_edge[1])] = count[int(next_edge[1])] + 1
        Final.append(sum(count)/5000)
        print(Final)
        save('Final a=0.1, b=0.1', Final)

[0.0]
[0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
